In [32]:
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
from datetime import datetime
from tqdm import tqdm

In [33]:
OCO2_filename = 'data/raw/OCO2_Europe_good_data.csv'
ICOS_filename = 'data/raw/ICOS_CO2_weather_euOct282023.csv'

# Load datasets
OCO2_df = pd.read_csv(OCO2_filename)
ICOS_df = pd.read_csv(ICOS_filename)


# drop quality flag now - since data is now quality verified
OCO2_df = OCO2_df.drop('xco2_quality_flag',axis=1)

# rename oco2 coords to lat and lon
OCO2_df = OCO2_df.rename(columns={"latitude": "lat", "longitude": "lon"})


#print(df.to_string())
print("OCO2", OCO2_df.columns)
print("ICOS", ICOS_df.columns)

OCO2 Index(['lat', 'lon', 'datetime', 'xco2'], dtype='object')
ICOS Index(['station', 'lat', 'lon', 'start_time', 'value', 'value_std_dev',
       '10m_u-component_of_wind', '10m_v-component_of_wind',
       'surface_pressure', 'temperature_2m', 'cloud_base_height',
       'total_cloud_cover', 'vertical_integral_of_temperature',
       'total_column_water_vapour'],
      dtype='object')


In [36]:
earliest_date = min(min(OCO2_df['datetime']), min(ICOS_df['start_time']))
print("Earliest Year:", int(earliest_date.split("-")[0]))

Earliest Year: 2014


In [37]:
def format_time(df, time_field):
    dates = []
    N =  len(df[time_field])
    for t in tqdm(df[time_field]):
        dates.append((datetime.strptime(t, "%Y-%m-%d %H:%M:%S") - datetime(2014, 1, 1)).total_seconds()/3600)
    df['hours_since_2014_1_1'] = dates
    return df

In [38]:
ICOS_df = format_time(ICOS_df, 'start_time')
OCO2_df = format_time(OCO2_df, 'datetime')

  0%|          | 0/685712 [00:00<?, ?it/s]

100%|██████████| 20847020/20847020 [02:24<00:00, 144612.41it/s]


In [39]:
os.makedirs("data/formatted", exist_ok=True)
ICOS_df.to_feather("data/formatted/ICOS_formatted.feather")
OCO2_df.to_feather("data/formatted/OCO2_formatted.feather")